In [1]:
import MeCab

f= open('pana_sdb2023j_IP.txt', encoding="utf-8")
text = f.read()
f.close

<function TextIOWrapper.close()>

In [2]:
#stopwordsの指定
with open("Japanese.txt","r") as f:
    stopwords1 = f.read().split("\n")

stopwords2 =["以下","ため","当社","当行","場合","影響","可能性","経営",
            "業績","状況","グループ","こと","平成","年","月","期","当","他",
            "等","お","これ","a","b","c","%","以上","もの","株式会社",
            "経営方針","もの","とも","ある","よる","的","化","*"]
stopwords = list(set(stopwords1+stopwords2))


import re
def mecab_tokenizer(text):

    replaced_text = text.lower()
    replaced_text = re.sub(r'[【】]', ' ', replaced_text)       # 【】の除去
    replaced_text = re.sub(r'[（）()]', ' ', replaced_text)     # （）の除去
    replaced_text = re.sub(r'[［］\[\]]', ' ', replaced_text)   # ［］の除去
    replaced_text = re.sub(r'[@＠]\w+', '', replaced_text)  # メンションの除去
    replaced_text = re.sub(r'\d+\.*\d*', '', replaced_text) #数字を0にする

    path = "-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
    mecab = MeCab.Tagger(path)
    parsed_lines = mecab.parse(replaced_text).split("\n")[:-2]
    
    # #表層形を取得
    surfaces = [l.split('\t')[0] for l in parsed_lines]
    
    #原形を取得
    token_list = [l.split("\t")[1].split(",")[6] for l in parsed_lines]
    #Chasenの場合
    #token_list = [l.split("\t")[2] for l in parsed_lines]
    
    #品詞を取得
    pos = [l.split('\t')[1].split(",")[0] for l in parsed_lines]
    #Chasenの場合
    #pos = [l.split('\t')[3].split("-")[0] for l in parsed_lines]
    
    # 名詞のみに絞り込み
    target_pos = ["名詞"]
    token_list = [t for t, p in zip(token_list, pos) if p in target_pos]
    
    # stopwordsの除去
    token_list = [t for t in token_list if t  not in stopwords]
    
    # ひらがなのみの単語を除く
    kana_re = re.compile("^[ぁ-ゖ]+$")
    token_list = [t for t in token_list if not kana_re.match(t)]
    
    return token_list

In [6]:
tokenized_text =mecab_tokenizer(text)
print(tokenized_text)

['知的財産', 'パナソニック', '方針', '責任者', '体制', '主', '取り組み', '社内', '教育', '社外', '啓発', '活動', '相談', '通報', '評価', '術', 'ノウハウ', 'デザイン', 'ブランド', 'その他', '成果', '知的財産', '適切', '取得', '保護', 'お呼び', '活用', '知的財産', '適切', '保護', 'お呼び', '活用', '当該', '知的財産', '模倣品', '侵害', '品等', '出現', '品質', '問題', '犯罪組織', '資金', '流入', '持続的', 'イノベーション', '阻害', '問題', '私たち', '第三者', '知的財産', '侵害', '当該', '第三者', '損失', '恐れ', '製品', 'サービス', '仕様変更', '供給', '中断', '直接', '間接', 'お客様', '迷惑', '恐れ', 'パナソニック', '事業活動', '第三者', '知的財産', '尊重', '知的財産', '事業化', '他社', '創', '社会', '実装', '事業', '成長', '社会', '課題', '解決', '実現', '方針', '創業', '以来', '事業', '知財', '精神', '事業', '知的財産', '起点', '戦略', '提案', 'グローバルな', '知的財産', '取得', '保護', '活用', 'お呼び', '知的財産', '紛争', '予防', '解決', '現在', '将来', '事業', '優位', '安全', '確保', '近年', '社会', '課題', '解決', '貢献', '視野', '知的財産', '活動', '推進', '着実', '遂行', '全体', '適用', '知的財産', '基本', '規程', '知的財産', '活動', '適切', '遂行', '活動', '基盤', '確立', '購入', '先様', '取引', '先様', 'その他', '第三者', '知的財産', '尊重', '侵害', '最善', 'パナソニックグループ', 'Compliance', '行動基準', '従業員', '全員', '順守', '定期', '教育', '責任者', 

In [7]:
import itertools
sentences = [mecab_tokenizer(sentence) for sentence in text.split("。")]
sentences_combs = [list(itertools.combinations(sentence,2)) for sentence in sentences]
words_combs = [[tuple(sorted(words)) for words in sentence] for sentence in sentences_combs]
target_combs = []
for words_comb in words_combs:
    target_combs.extend(words_comb)
 
import collections
ct = collections.Counter(target_combs)

import pandas as pd
df = pd.DataFrame([{"1番目" : i[0][0], "2番目": i[0][1], "count":i[1]} for i in ct.most_common()])
df.head(30)

,1番目,2番目,count
0,活動,知的財産,23
1,知的財産,知的財産,22
2,事業,知的財産,18
3,件,件数,18
4,件,保有,18
5,件,内海,18
6,知的財産,社会,17
7,パナソニック,知的財産,16
8,知的財産,解決,15
9,戦略,知的財産,15


In [9]:
import itertools
sentences = [mecab_tokenizer(sentence) for sentence in text.split("。")]
print(sentences)
sentences_combs = [list(itertools.combinations(sentence,2)) for sentence in sentences]
words_combs = [[tuple(sorted(words)) for words in sentence] for sentence in sentences_combs]
target_combs = []
for words_comb in words_combs:
    target_combs.extend(words_comb)
 
import collections
ct = collections.Counter(target_combs)

import pandas as pd
df = pd.DataFrame([{"1番目" : i[0][0], "2番目": i[0][1], "count":i[1]} for i in ct.most_common()])
df.head(30)

[['知的財産', 'パナソニック', '方針', '責任者', '体制', '主', '取り組み', '社内', '教育', '社外', '啓発', '活動', '相談', '通報', '評価', '術', 'ノウハウ', 'デザイン', 'ブランド', 'その他', '成果', '知的財産', '適切', '取得', '保護', 'お呼び', '活用'], ['知的財産', '適切', '保護', 'お呼び', '活用', '当該', '知的財産', '模倣品', '侵害', '品等', '出現', '品質', '問題', '犯罪組織', '資金', '流入', '持続的', 'イノベーション', '阻害', '問題'], ['私たち', '第三者', '知的財産', '侵害', '当該', '第三者', '損失', '恐れ', '製品', 'サービス', '仕様変更', '供給', '中断', '直接', '間接', 'お客様', '迷惑', '恐れ'], ['パナソニック', '事業活動', '第三者', '知的財産', '尊重', '知的財産', '事業化', '他社', '創', '社会', '実装', '事業', '成長', '社会', '課題', '解決', '実現'], ['方針', '創業', '以来', '事業', '知財', '精神', '事業', '知的財産', '起点', '戦略', '提案', 'グローバルな', '知的財産', '取得', '保護', '活用', 'お呼び', '知的財産', '紛争', '予防', '解決', '現在', '将来', '事業', '優位', '安全', '確保', '近年', '社会', '課題', '解決', '貢献', '視野', '知的財産', '活動', '推進'], ['着実', '遂行', '全体', '適用', '知的財産', '基本', '規程', '知的財産', '活動', '適切', '遂行', '活動', '基盤', '確立'], ['購入', '先様', '取引', '先様', 'その他', '第三者', '知的財産', '尊重', '侵害', '最善'], ['パナソニックグループ', 'Compliance', '行動基準', '従業員', '全員', '順守', '定期'

,1番目,2番目,count
0,活動,知的財産,23
1,知的財産,知的財産,22
2,事業,知的財産,18
3,件,件数,18
4,件,保有,18
5,件,内海,18
6,知的財産,社会,17
7,パナソニック,知的財産,16
8,知的財産,解決,15
9,戦略,知的財産,15


In [7]:
def kyoki_word_network(df):
    from pyvis.network import Network
    import pandas as pd

    
    got_net = Network(height="1000px", width="95%", bgcolor="#FFFFFF", font_color="black", notebook=True)

    got_net.force_atlas_2based()
    got_data = df[:300]

    sources = got_data['1番目']#count
    targets = got_data['2番目']#first
    weights = got_data['count']#second

    edge_data = zip(sources, targets, weights)

    for e in edge_data:
        src = e[0]
        dst = e[1]
        w = e[2]

        got_net.add_node(src, src, title=src)
        got_net.add_node(dst, dst, title=dst)
        got_net.add_edge(src, dst, value=w)

    neighbor_map = got_net.get_adj_list()

    for node in got_net.nodes:
        node["title"] += " Neighbors:<br>" + "<br>".join(neighbor_map[node["id"]])
        node["value"] = len(neighbor_map[node["id"]])

    got_net.show_buttons(filter_=['physics'])
    return got_net

In [8]:
got_net = kyoki_word_network(df)
got_net.show("kyoki.html")